# Model training

In [ ]:
import os

SPRITE_DIRECTORY = 'sprites'
TRAIN_DATA_DIRECTORY = os.path.join(SPRITE_DIRECTORY, 'training')
TEST_DATA_DIRECTORY = os.path.join(SPRITE_DIRECTORY, 'test')

## Set up data flow

In [ ]:
import tensorflow

IMAGE_SIZE = 128
BATCH_SIZE = 32

print(f'Flowing from {TRAIN_DATA_DIRECTORY}')
varying_image_generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.05
)
train_image_generator = varying_image_generator.flow_from_directory(
    directory=TRAIN_DATA_DIRECTORY,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    batch_size=BATCH_SIZE,
    color_mode='rgba',
    save_to_dir='/generated'
)
print(f'Flowing from {TEST_DATA_DIRECTORY}')
constant_image_generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
test_image_generator = constant_image_generator.flow_from_directory(
    directory=TEST_DATA_DIRECTORY,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=False,
    color_mode='rgba',
    batch_size=BATCH_SIZE
)

## Count data

In [ ]:
import os

def get_file_count_in_subdirectories_of(directory):
    subdirectories = os.listdir(directory)
    sum = 0
    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(directory, subdirectory)
        files = os.listdir(subdirectory_path)
        file_count = len(files)
        sum = sum + file_count
    print(f'Found {sum} files in {directory}')
    return sum

TRAIN_DATA_COUNT = get_file_count_in_subdirectories_of(TRAIN_DATA_DIRECTORY)
TEST_DATA_COUNT = get_file_count_in_subdirectories_of(TEST_DATA_DIRECTORY)

## View data

In [ ]:
import matplotlib.pyplot as plot
import numpy

sample_training_images, labels = next(test_image_generator)
class_label = {v:k for k,v in test_image_generator.class_indices.items()}
fig, axes = plot.subplots(4, 5, figsize=(10, 10))
axes = axes.flatten()
for img, label, ax in zip(sample_training_images, labels, axes):
    ax.set_title(class_label[numpy.argmax(label)])
    ax.imshow(img)
    ax.axis('off')
plot.tight_layout()

## Build network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 4)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(2, activation='sigmoid')
])

The model now looks like this

In [ ]:
model.summary()

## Train model

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

snapshot_callback = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath="./.snapshots"
)

EPOCHS = 200
STEPS = TRAIN_DATA_COUNT / BATCH_SIZE
VALIDATION_STEPS = TEST_DATA_COUNT / BATCH_SIZE

history = model.fit_generator(
    steps_per_epoch=STEPS,
    epochs=EPOCHS,
    generator=train_image_generator,
    #callbacks=[snapshot_callback],
    validation_data=test_image_generator,
    validation_steps=VALIDATION_STEPS
)
print('Done training')

## Evaluate score

In [ ]:
import numpy
from sklearn import metrics

prediction = model.predict(test_image_generator)
prediction = numpy.argmax(prediction, axis=1)
images, actual = next(test_image_generator)
actual = numpy.argmax(actual, axis=1)
print(actual)
score = metrics.accuracy_score(actual, prediction)
print(f'Final Score: {score}')

## Monitor training

In [ ]:
import matplotlib.pyplot as plot

LOSS = history.history['loss']
LOSS_VALIDATION = history.history['val_loss']

ACCURACY = history.history['accuracy']
ACCURACY_VALIDATION = history.history['val_accuracy']

EPOCHS_RANGE = range(EPOCHS)

plot.figure()
plot.subplot(1, 2, 1)
plot.plot(EPOCHS_RANGE, LOSS, label='Training')
plot.plot(EPOCHS_RANGE, LOSS_VALIDATION, label='Test')
plot.legend()
plot.title('Loss')

plot.subplot(1, 2, 2)
plot.plot(EPOCHS_RANGE, ACCURACY, label='Training')
plot.plot(EPOCHS_RANGE, ACCURACY_VALIDATION, label='Test')
plot.legend()
plot.title('Accuracy')

plot.show()

## Confusion matrix

Because the dataset is not balanced (i.e. there are more water than grass sprites), the values in the confusion matrix need to be normalized. Otherwise, the calculation will be biased towards the water sprites.

In [ ]:
import matplotlib.pyplot as plot
from sklearn import metrics
import numpy

class_names = ['grass', 'water']
confusion_matrix = metrics.confusion_matrix(actual, prediction)
normalized_confusion_matrix = confusion_matrix / confusion_matrix.sum(axis=1)[:, numpy.newaxis]
plot.imshow(normalized_confusion_matrix)
plot.title('Confusion matrix')
plot.colorbar()
tick_marks = [0, 1]
plot.xticks(tick_marks, class_names, rotation=45)
plot.yticks(tick_marks, class_names)
plot.ylabel('Actual')
plot.xlabel('Predicted')

## Plot ROC

In [ ]:
import matplotlib.pyplot as plot
import sklearn.metrics

predictions = model.predict(test_image_generator)
positive_predictions = predictions[:, 1]

fpr, tpr, _ = metrics.roc_curve(actual, positive_predictions)
roc_auc = metrics.auc(fpr, tpr)

plot.title(f'ROC (area = {roc_auc})')
plot.plot(fpr, tpr)
plot.plot([0, 1], [0, 1], ':k')
plot.xlabel('False positive')
plot.ylabel('True positive')

## Save model

In [ ]:
model.save('pokemon_type_model.h5')